In [14]:
import math
import geopandas as gpd
import pandas as pd
import pydeck as pdk
import numpy as np
import copy
from itertools import chain

In [15]:
alt = gpd.read_file('/Users/nicolesalazar/Documents/Properati/Altura/Quito/UIO_ALTURA.geojson')
alt

,id,elevation,Pop_count,Pop_sum,km_count,km_sum,Valor del m²_mean,casas_Valor del m²_mean,spa_nam_place,Zona-Nombre,geometry
0,92,3368,NaN,NaN,NaN,NaN,NaN,NaN,Protección Natural Chillogallo,Sur de Quito,"MULTIPOLYGON (((-78.60865 -0.28178, -78.60829 ..."
1,93,3405,NaN,NaN,NaN,NaN,NaN,NaN,Protección Natural Chillogallo,Sur de Quito,"MULTIPOLYGON (((-78.61043 -0.28627, -78.61053 ..."
2,94,3398,NaN,NaN,NaN,NaN,NaN,NaN,Protección Natural Chillogallo,Sur de Quito,"MULTIPOLYGON (((-78.60992 -0.29077, -78.60992 ..."
3,95,3498,NaN,NaN,NaN,NaN,NaN,NaN,Protección Natural Chillogallo,Sur de Quito,"MULTIPOLYGON (((-78.61043 -0.29526, -78.61037 ..."
4,96,3629,NaN,NaN,NaN,NaN,NaN,NaN,Protección Natural Ecuatoriana,Sur de Quito,"MULTIPOLYGON (((-78.61067 -0.29975, -78.61091 ..."
...,...,...,...,...,...,...,...,...,...,...,...
3750,7803,2519,NaN,NaN,NaN,NaN,NaN,NaN,Puembo,Valle Tumbaco,"MULTIPOLYGON (((-78.34141 -0.19699, -78.34143 ..."
3751,7804,2527,NaN,NaN,NaN,NaN,NaN,NaN,Puembo,Valle Tumbaco,"MULTIPOLYGON (((-78.34141 -0.20263, -78.34196 ..."
3752,7933,2461,NaN,NaN,NaN,NaN,NaN,NaN,Puembo,Valle Tumbaco,"MULTIPOLYGON (((-78.34141 -0.19628, -78.34140 ..."
3753,7934,2485,NaN,NaN,NaN,NaN,NaN,NaN,Puembo,Valle Tumbaco,"MULTIPOLYGON (((-78.34130 -0.19644, -78.34128 ..."


In [16]:
cero=alt['geometry'].values

In [4]:
#Pasa Multipolygon a Coordenadas
def to_coords(multipolygon):
    for polygon in multipolygon:
        yield from polygon.exterior.coords[:-1]
        yield from chain.from_iterable(interior.coords[:-1] for interior in polygon.interiors)

In [5]:
lpoly=[]
for i in range(len(cero)):
  lpoly.append([])
  for j in range(1):
    lpoly[i].append(list(to_coords(cero[i])))	

In [6]:
coor=lpoly
coor=np.array(coor)
alt['coordinates1']=coor

<ipython-input-6-d9191f8df0f1>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  coor=np.array(coor)


In [7]:
alt['elevation'	].max()

4678

In [8]:
# Color Magenta
COLOR_RANGE1 = [
    [8, 24, 37],
    [41, 60, 75],
    [60, 93, 118], 
    [83, 115, 131],
    [86, 110, 96],    
    [115, 133, 94], 
    [177, 182, 126],
    [189, 163, 110],    
    [243, 200, 152],
    [246, 214, 196],
 #mas oscuro
]

#('#383740','#234f5e','#008490','#8dc78a','#c2c2ca','#f5b6c3','#f6d598')
BREAKS1 = [1600,2000,2200,2500,2700,3000,3500,3800, 4000,4600,]


def color_scale1(val):
    for i, b in enumerate(BREAKS1):
        if val < b:
            return COLOR_RANGE1[i]
    return COLOR_RANGE1[i]

In [9]:
alt['color']=alt['elevation'].apply(color_scale1)

In [12]:
alt['elevation'].max()

4678

In [17]:
# Add sunlight shadow to the polygons
#sunlight = {
#    "@@type": "_SunLight",
#    "timestamp": 1564696800000,  # Date.UTC(2019, 7, 1, 22),
#    "color": [255, 255, 255],
#    "intensity": 1.0,
#    "_shadow": True,
#}

#ambient_light = {"@@type": "AmbientLight", "color": [255, 255, 255], "intensity": 1.0}

#lighting_effect = {
#    "@@type": "LightingEffect",
#    "shadowColor": [0, 0, 0, 0.2],
#    "ambientLight": ambient_light,
#    "directionalLights": [sunlight],
#}

view_state = pdk.ViewState(
    **{"latitude": -0.170735, "longitude":  -78.467342, "zoom": 10, "maxZoom": 16, "pitch": 45, "bearing": 0}
)

LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

polygon_layer = pdk.Layer(
    "PolygonLayer",
    LAND_COVER,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon="-",
    get_fill_color=[0, 0, 0, 20],
)

polygon_layer = pdk.Layer(
    "PolygonLayer",
    alt,
    id="geojson",
    opacity=0.6,
    stroked=False,
    get_polygon="coordinates1",
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation="(elevation/3000)*elevation*3", #Población
    get_fill_color="color", #Precio m2
    get_line_color='color',
    auto_highlight=True,
    pickable=True,
    coverage=1,
)

tooltip = {"html": "<b>Ubicación:</b> {Zona-Nombre}, {spa_nam_place} <br /><b>Elevación:</b> {elevation}  <br /><b>Población:</b> {Pop_sum}<br /> <b>Precio del M² (USD):</b> {Valor del m²_mean}"}

r = pdk.Deck(
    polygon_layer,
    initial_view_state=view_state,
    map_style=pdk.map_styles.LIGHT,
    tooltip=tooltip,
)
r.to_html('Quito_altura1.html')

In [12]:
r

NameError: name 'r' is not defined